# SCFEM

In [ ]:
class SCFEM(nn.Module):
    def __init__(self, c1, c2, n=1, shortcut=True:  
        super(SCFEM, self).__init__()

        #Apply padding to dilated convolutions to keep output size consistent
        self.dilated_conv1 = nn.Conv2d(c1, c2, kernel_size=3, dilation=1, padding=1)
        self.dilated_conv3 = nn.Conv2d(c1, c2, kernel_size=3, dilation=3, padding=3)
        self.dilated_conv5 = nn.Conv2d(c1, c2, kernel_size=3, dilation=5, padding=5)
        self.dilated_conv7 = nn.Conv2d(c1, c2, kernel_size=3, dilation=7, padding=7)

        self.batch_norm = nn.BatchNorm2d(c2 * 4)  # Update to c2 * 4
        self.relu = nn.ReLU()

        # 1x1 convolution to match the number of channels for the shortcut connection
        self.conv1x1_shortcut = nn.Conv2d(c1, c2 * 4, kernel_size=1)

        # CBL layers for the final output
        self.conv_final1 = nn.Conv2d(c2 * 4, c2, kernel_size=1)
        self.batch_norm_final1 = nn.BatchNorm2d(c2)  # Update to c2
        self.leaky_relu1 = nn.LeakyReLU(0.1)

    def forward(self, x):
        out1 = self.dilated_conv1(x)
        out2 = self.dilated_conv3(x)
        out3 = self.dilated_conv5(x)
        out4 = self.dilated_conv7(x)

        shortcut = self.conv1x1_shortcut(x)  # 1x1 convolution for shortcut connection
        shortcut = F.interpolate(shortcut, size=(out1.size(2), out1.size(3)), mode='bilinear', align_corners=False)

        concat_input = torch.cat([out1, out2, out3, out4], dim=1)
        output = concat_input + shortcut
        output = self.batch_norm(output)
        output = self.relu(output)

        # Applying CBL to the concatenated output
        output = self.conv_final1(output)
        output = self.batch_norm_final1(output)
        output = self.leaky_relu1(output)

        # Upsampling the output to the original size
        output = F.interpolate(output, size=(x.size(2), x.size(3)), mode='bilinear', align_corners=False)

        return output


# SAFFM

In [ ]:
class SAFFM(nn.Module):
    def __init__(self, c1, c2, k=5):
        super(SAFFMModule, self).__init__()

        # 1x1 Convolution to reduce channels and apply Batch Normalization
        self.conv1x1 = nn.Sequential(
            nn.Conv2d(c1, c2, kernel_size=1),
            nn.BatchNorm2d(c2),
            nn.LeakyReLU(0.1)
        )

        # Max Pooling layers with different kernel sizes
        self.maxpool_5x5 = nn.MaxPool2d(kernel_size=k, stride=1, padding=k//2)
        self.maxpool_9x9 = nn.MaxPool2d(kernel_size=k*2, stride=1, padding=k)
        self.maxpool_13x13 = nn.MaxPool2d(kernel_size=k*3, stride=1, padding=k*2)

        # 1x1 Convolution to adjust channels after concatenation
        self.concat_conv1 = nn.Conv2d(c2 * 4, c2, kernel_size=1)
        self.concat_conv2 = nn.Conv2d(c2 * 2, c2, kernel_size=1)

        # Final 1x1 Convolution to get 1024 output channels
        self.final_conv = nn.Conv2d(c2, 1024, kernel_size=1)

    def forward(self, x):
        # Reduce channels and apply Batch Normalization
        x = self.conv1x1(x)

        # Max Pooling layers
        pool_5x5 = self.maxpool_5x5(x)
        pool_9x9 = self.maxpool_9x9(x)
        pool_13x13 = self.maxpool_13x13(x)

        # Concatenate all the pooling outputs
        concat_output = torch.cat([x, pool_5x5, pool_9x9, pool_13x13], dim=1)

        # Apply 1x1 Convolution to reduce channels
        concat_output = self.concat_conv1(concat_output)

        # Shortcut connection
        shortcut = self.conv1x1(x)

        # Concatenate shortcut connection to the current Concatenation layer
        concat_output = torch.cat([concat_output, shortcut], dim=1)

        # Apply another 1x1 Convolution to reduce channels to 1024
        concat_output = self.concat_conv2(concat_output)

        # Apply final 1x1 Convolution to get 1024 output channels
        output = self.final_conv(concat_output)

        return output

# BINSNet

In [ ]:
class BINOBlock(nn.Module):
    def __init__(self, c1, c2, shortcut=True, g=1, e=0.5):
        super(BINOBlock, self).__init__()

        # Inside BINO Block
        self.conv1x1 = nn.Conv2d(c1, c2, kernel_size=1, stride=1, padding=0)
        self.batch_norm1x1 = nn.BatchNorm2d(c2)

        # Spatially Separate Convolution (3x3 and 5x5)
        self.conv3x3 = nn.Conv2d(c2, c2, kernel_size=3, stride=1, padding=1)
        self.conv5x5 = nn.Conv2d(c2, c2, kernel_size=5, stride=1, padding=2)
        self.batch_norm3x3_5x5 = nn.BatchNorm2d(c2)

        # Up-sampling to Spatially Separate Convolution (5x5 and 7x7)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

        # Spatially Separate Convolution (5x5 and 7x7)
        self.conv5x5_upsampled = nn.Conv2d(c2, c2, kernel_size=5, stride=1, padding=2)
        self.conv7x7_upsampled = nn.Conv2d(c2, c2, kernel_size=7, stride=1, padding=3)
        self.batch_norm5x5_7x7 = nn.BatchNorm2d(c2)

        # Up-sampling to 3x3
        self.upsample_3x3 = nn.Upsample(scale_factor=2, mode='nearest')

        # 1x1 Convolution followed by Activation Function (Leaky ReLU)
        self.conv1x1_leaky_relu = nn.Conv2d(c2, c2, kernel_size=1, stride=1, padding=0)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv1x1(x)
        x = self.batch_norm1x1(x)

        # Spatially Separate Convolution (3x3 and 5x5)
        x3x3 = self.conv3x3(x)
        x5x5 = self.conv5x5(x)
        x = torch.cat([x3x3, x5x5], dim=1)
        x = self.batch_norm3x3_5x5(x)

        # Up-sampling to Spatially Separate Convolution (5x5 and 7x7)
        x = self.upsample(x)
        x5x5_upsampled = self.conv5x5_upsampled(x)
        x7x7_upsampled = self.conv7x7_upsampled(x)
        x = torch.cat([x5x5_upsampled, x7x7_upsampled], dim=1)
        x = self.batch_norm5x5_7x7(x)

        # Up-sampling to 3x3
        x = self.upsample_3x3(x)

        # 1x1 Convolution followed by Activation Function (Leaky ReLU)
        x = self.conv1x1_leaky_relu(x)
        x = self.leaky_relu(x)

        return x

class BINSNet(nn.Module):
    # BINSNet with additional parameters
    def __init__(self, c1, c2, shortcut=True, g=1, e=0.5):
        super(BINSNet, self).__init__()

        # 1x1 Convolution followed by Batch Normalization
        self.conv1x1_1 = nn.Conv2d(c1, c2, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(c2)

        # 3x3 Convolution followed by Batch Normalization
        self.conv3x3 = nn.Conv2d(c2, c2, kernel_size=3, stride=1, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(c2)

        # BINO Block
        self.bino_block = BINOBlock(c2, c2)

        # 3x3 Convolution followed by Batch Normalization
        self.conv3x3_2 = nn.Conv2d(c2, c2, kernel_size=3, stride=1, padding=1)
        self.batch_norm3_2 = nn.BatchNorm2d(c2)

        # 1x1 Convolution
        self.conv1x1_2 = nn.Conv2d(c2, c2, kernel_size=1, stride=1, padding=0)

        # Additional 3x3 Convolution with Batch Normalization
        self.additional_conv3x3 = nn.Conv2d(c2 * 2, c2, kernel_size=3, stride=1, padding=1)
        self.additional_batch_norm3 = nn.BatchNorm2d(c2)

    def forward(self, x, prev_module_output):
        # 1x1 Convolution followed by Batch Normalization
        x = self.conv1x1_1(x)
        x = self.batch_norm1(x)

        # 3x3 Convolution followed by Batch Normalization
        x = self.conv3x3(x)
        x = self.batch_norm3(x)

        # BINO Block
        bino_output = self.bino_block(x)

        # 3x3 Convolution followed by Batch Normalization
        x = self.conv3x3_2(bino_output)
        x = self.batch_norm3_2(x)

        # Concatenation
        x = torch.cat([bino_output, x, prev_module_output], dim=1)

        # Additional 3x3 Convolution with Batch Normalization
        x = self.additional_conv3x3(x)
        x = self.additional_batch_norm3(x)

        # 1x1 Convolution
        x = self.conv1x1_2(x)

        return x

# DBINSNet

In [ ]:
class DBINSNet(nn.Module):
    # DBINSNet with additional parameters
    def __init__(self, c1, c2, shortcut=True, g=1, e=0.5):
        super(DBINSNet, self).__init__()

        # 1x1 Convolution followed by Batch Normalization
        self.conv1x1_1 = nn.Conv2d(c1, c2, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(c2)

        # BINO Blocks (Repeat 3 times)
        self.dbino_block1 = BINOBlock(c2, c2)
        self.dbino_block2 = BINOBlock(c2, c2)
        self.dbino_block3 = BINOBlock(c2, c2)

        # 1x1 Convolution followed by Batch Normalization
        self.conv1x1_2 = nn.Conv2d(c2, c2, kernel_size=1, stride=1, padding=0)
        self.batch_norm2 = nn.BatchNorm2d(c2)

        # Shortcut connection
        self.shortcut_conv1x1 = nn.Conv2d(c1, c2, kernel_size=1, stride=1, padding=0)
        self.batch_norm_shortcut = nn.BatchNorm2d(c2)

        # Additional 1x1 Convolution
        self.additional_conv1x1 = nn.Conv2d(c2, c2, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        # 1x1 Convolution followed by Batch Normalization
        x = self.conv1x1_1(x)
        x = self.batch_norm1(x)

        # BINO Blocks
        bino_output1 = self.bino_block1(x)
        bino_output2 = self.bino_block2(bino_output1)
        bino_output3 = self.bino_block3(bino_output2)

        # Concatenation
        x = torch.cat([bino_output1, bino_output2, bino_output3, x], dim=1)
        
        # 1x1 Convolution followed by Batch Normalization
        x = self.conv1x1_2(x)
        x = self.batch_norm2(x)

        # Shortcut connection
        shortcut = self.shortcut_conv1x1(x)
        shortcut = self.batch_norm_shortcut(shortcut)

        # Additional 1x1 Convolution
        x = self.additional_conv1x1(x)

        return x + shortcut  # Shortcut connection (Residual connection)

